# Prepare CNN Data

## Setup

In [1]:
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AX4XfWiklQyHFnYZILj0DDqebsJG86MrlEFcWXUdQjgIbfC7JNSZxHC8mBA



Successfully saved authorization token.


In [2]:
import numpy as np
#import geetools
#from geetools import ui, cloud_mask
import os, datetime
import pandas as pd
import itertools
import tensorflow as tf
import config as cf
import ee_utils as utils

#cloud_mask_landsatSR = cloud_mask.landsatSR()
#cloud_mask_sentinel2 = cloud_mask.sentinel2()

/Users/robmarty/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/robmarty/anaconda3/lib/python3.7/site-packages/xarray/core/merge.py:17: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)
/Users/robmarty/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


## Parameters

In [3]:
KERNEL_SIZE = 224
SURVEY_NAME = 'DHS'
SATELLITE = 's2'
SKIP_IF_SCRAPED = True
CHUNK_SIZE = 10 # Number of observtaions to scrape in GEE at any given time
PROJECT_DIR = cf.GOOGLEDRIVE_DIRECTORY

In [4]:
# Directory to store tfrecords
out_path = os.path.join(PROJECT_DIR, 
            'Data', 
            SURVEY_NAME, 
            'FinalData',
            'Individual Datasets',
            'cnn_' + SATELLITE,
            'tfrecords')

## Implement

In [5]:
## Load Survey
survey_df = pd.read_csv(os.path.join(PROJECT_DIR, 'Data', SURVEY_NAME, 'FinalData', 'Individual Datasets', 'survey_socioeconomic.csv'))
#survey_df = survey_df[survey_df.country_code == 'PK']
tf_record_list = list(np.unique(survey_df.tfrecord_name))

/Users/robmarty/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (27,31,35) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
# Remove if issues extracting
survey_df = survey_df[survey_df['uid'] != "IA201400180079"]
survey_df = survey_df[survey_df['uid'] != "IA201400180052"]
survey_df = survey_df[survey_df['uid'] != "IA201400180112"]
survey_df = survey_df[survey_df['uid'] != "IA201400180081"]
survey_df = survey_df[survey_df['uid'] != "IA201400180011"]
survey_df = survey_df[survey_df['uid'] != "IA201400180048"]
survey_df = survey_df[survey_df['uid'] != "IA201400180058"]
survey_df = survey_df[survey_df['uid'] != "IA201400180028"]
survey_df = survey_df[survey_df['uid'] != "IA201400180072"]
survey_df = survey_df[survey_df['uid'] != "IA201400180047"]
survey_df = survey_df[survey_df['uid'] != "IA201400180012"]
survey_df = survey_df[survey_df['uid'] != "IA201400180040"]

In [17]:
# If skip already scraped, remove existing tfrecords from tf_record_list
if SKIP_IF_SCRAPED:
    tf_records_exist = os.listdir(out_path)
    tf_record_list = [x for x in tf_record_list if x not in tf_records_exist]
    


In [ ]:
### Loop through all tfrecords
for tfr_i in tf_record_list:
    
    survey_df_yeari = survey_df[survey_df['tfrecord_name'] == tfr_i]
    year_i = survey_df_yeari['year'].iloc[0]

    ### Loop through chunks within tfrecord (can only pull so much data from GEE at a time)
    survey_df_yeari['chunk_id'] = utils.chunk_ids(survey_df_yeari.shape[0], CHUNK_SIZE)

    print("Putting " + str(survey_df_yeari.shape[0]) + " observations into " + tfr_i)
    
    proto_examples_all = []
    for chunk_i in list(np.unique(survey_df_yeari.chunk_id)):
        
        print("Observation: " + str(len(proto_examples_all)) + "/" + str(survey_df_yeari.shape[0]))
        
        survey_df_yeari_chunki = survey_df_yeari[survey_df_yeari['chunk_id'] == chunk_i]
      
        proto_examples_i = utils.prep_cnn_np(survey_df_yeari_chunki, SATELLITE, KERNEL_SIZE, year_i)
        proto_examples_all.extend(proto_examples_i)

    ### Save data as tf record
    out_path_i = os.path.join(out_path, tfr_i)
    print(out_path_i)
    with tf.io.TFRecordWriter(out_path_i) as writer:
        for tf_example in proto_examples_all:
            writer.write(tf_example.SerializeToString())

/Users/robmarty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Putting 198 observations into IA_2_19.tfrecord
Observation: 0/198
Observation: 10/198
Observation: 20/198
Observation: 30/198
Observation: 40/198
Observation: 50/198
Observation: 60/198
Observation: 70/198
Observation: 80/198
Observation: 90/198
Observation: 100/198
Observation: 110/198
Observation: 120/198
Observation: 130/198
Observation: 140/198
Observation: 150/198
Observation: 160/198
Observation: 170/198
Observation: 180/198
Observation: 190/198
/Users/robmarty/Google Drive/World Bank/IEs/Pakistan Poverty Estimation/Data/DHS/FinalData/Individual Datasets/cnn_s2/tfrecords/IA_2_19.tfrecord
Putting 200 observations into IA_2_2.tfrecord
Observation: 0/200
Observation: 10/200
Observation: 20/200
Observation: 30/200
Observation: 40/200
Observation: 50/200
Observation: 60/200
Observation: 70/200
Observation: 80/200
Observation: 90/200
Observation: 100/200
Observation: 110/200
Observation: 120/200
Observation: 130/200
Observation: 140/200
Observation: 150/200
Observation: 160/200
Observa

In [ ]:
for row_i in range(0,10):
    print(row_i)
    proto_examples_i = utils.prep_cnn_np(survey_df_yeari_chunki.iloc[[row_i]], SATELLITE, KERNEL_SIZE, year_i)

In [15]:
survey_df_yeari_chunki.iloc[[7]]

,gadm_uid,uid,country_code,country_year,urban_rural,year,most_recent_survey,latitude,longitude,cluster_id,...,NAME_2,VARNAME_2,NL_NAME_2,TYPE_2,ENGTYPE_2,CC_2,HASC_2,within_country_fold,tfrecord_name,chunk_id
11998,289,IA201400180040,IA,IA_2015_16,U,2015,True,10.562917,72.617523,180040,...,Lakshadweep,NaN,NaN,District,District,NaN,IN.LD.LD,IA_2,IA_2_19.tfrecord,9
